In [55]:
import os
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import shutil
import nibabel as nib
import numpy as np
from skimage import exposure
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, UpSampling2D

In [56]:
datasets_folder = "Datasets"
labels_folder = os.path.join(datasets_folder, "Labels")
scans_folder = os.path.join(datasets_folder, "Scans")
output_folder = os.path.join(datasets_folder, "Data")

# Create the output folder
os.makedirs(output_folder, exist_ok=True)

# Copy scan files to the output folder
for file in os.listdir(scans_folder):
    if file.endswith('.nii.gz'):
        shutil.copy(os.path.join(scans_folder, file), output_folder)

# Copy label files to the output folder
for file in os.listdir(labels_folder):
    if file.endswith('.nii.gz'):
        shutil.copy(os.path.join(labels_folder, file), output_folder)

print("Data copied successfully to the output folder:", output_folder)


Data copied successfully to the output folder: Datasets/Data


In [57]:
# Step 3: Split the dataset
scan_files = [file for file in os.listdir(output_folder) if file.endswith('.nii.gz')]
train_files, val_test_files = train_test_split(scan_files, test_size=0.2, random_state=42)
val_files, test_files = train_test_split(val_test_files, test_size=0.5, random_state=42)

train_folder = os.path.join(output_folder, 'train')
val_folder = os.path.join(output_folder, 'val')
test_folder = os.path.join(output_folder, 'test')

# Create train, validation, and test folders
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Move train files
for file in train_files:
    shutil.move(os.path.join(output_folder, file), os.path.join(train_folder, file))

# Move validation files
for file in val_files:
    shutil.move(os.path.join(output_folder, file), os.path.join(val_folder, file))

# Move test files
for file in test_files:
    shutil.move(os.path.join(output_folder, file), os.path.join(test_folder, file))

print("Dataset split successfully.")

Dataset split successfully.


In [58]:
def unet_model(input_shape, num_classes):
    inputs = Input(input_shape)

    # Contracting Path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottom
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Expanding Path
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = Conv2D(num_classes, 3, activation='softmax', padding='same')(conv9)

    model = Model(inputs=inputs, outputs=conv9)
    return model

# Define the input shape and number of classes
input_shape = (256, 256, 1)  
num_classes = 3  # Background, Pancreas, Tumor

# Create the U-Net model
model = unet_model(input_shape, num_classes)


In [78]:
# Step 5: Train the Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data directories
train_folder = 'Datasets/Data/train'
val_folder = 'Datasets/Data/val'
test_folder = 'Datasets/Data/test'

# Define the image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Define the batch size
batch_size = 32


# Load the training data
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode=None,
    shuffle=True
)

# Load the validation data
val_generator = val_datagen.flow_from_directory(
    val_folder,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode=None,
    shuffle=True
)

# Load the test data
test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

# Calculate the number of steps per epoch
steps_per_epoch = len(train_generator)
validation_steps = len(val_generator)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=validation_steps
)


Found 0 images belonging to 0 classes.
python
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: Asked to retrieve element 0, but the Sequence has length 0